In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import pickle

print("Loading datasets...")
train_df = pd.read_excel('../data/raw/train.xlsx')
test_df = pd.read_excel('../data/raw/test.xlsx')

print(f"Train shape: {train_df.shape}")
print(f"Test shape: {test_df.shape}")
print(f"\nColumns: {list(train_df.columns)}")


Loading datasets...
Train shape: (16209, 21)
Test shape: (5404, 20)

Columns: ['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15']


In [2]:
def engineer_features(df, is_train=True):
    """Create powerful engineered features"""
    
    df = df.copy()
    
    # 1. Price per sqft metrics (only for train, target variable)
    if is_train:
        df['price_per_sqft'] = df['price'] / df['sqft_living']
        df['price_per_lot'] = df['price'] / df['sqft_lot']
    
    # 2. Neighborhood comparison features (VERY IMPORTANT)
    df['living_vs_neighbors'] = df['sqft_living'] / (df['sqft_living15'] + 1)
    df['lot_vs_neighbors'] = df['sqft_lot'] / (df['sqft_lot15'] + 1)
    
    # 3. Basement features
    df['has_basement'] = (df['sqft_basement'] > 0).astype(int)
    df['basement_ratio'] = df['sqft_basement'] / df['sqft_living']
    
    # 4. Quality composite score
    df['quality_score'] = df['grade'] * df['condition']
    df['luxury_score'] = df['grade'] + df['condition'] + df['view'] + df['waterfront']
    
    # 5. Age features
    df['age'] = 2015 - df['yr_built']  # Dataset from 2015
    df['years_since_renovation'] = 2015 - df['yr_renovated']
    df['years_since_renovation'] = df['years_since_renovation'].replace(2015, 0)
    df['is_renovated'] = (df['yr_renovated'] > 0).astype(int)
    
    # 6. Space utilization
    df['bedrooms_per_sqft'] = df['bedrooms'] / df['sqft_living']
    df['bathrooms_per_bedroom'] = df['bathrooms'] / (df['bedrooms'] + 1)
    
    # 7. Above ground ratio
    df['above_ground_ratio'] = df['sqft_above'] / df['sqft_living']
    
    # 8. Lot coverage (building footprint on lot)
    df['lot_coverage'] = df['sqft_living'] / df['sqft_lot']
    
    # 9. Premium features
    df['has_view'] = (df['view'] > 0).astype(int)
    df['high_grade'] = (df['grade'] >= 9).astype(int)
    df['excellent_condition'] = (df['condition'] >= 4).astype(int)
    
    # 10. Geographic features - Location clusters
    coords = df[['lat', 'long']].values
    kmeans = KMeans(n_clusters=10, random_state=42, n_init=10)
    df['location_cluster'] = kmeans.fit_predict(coords)
    
    # 11. Distance from city center (Seattle downtown: 47.6062, -122.3321)
    seattle_lat, seattle_long = 47.6062, -122.3321
    df['dist_from_downtown'] = np.sqrt(
        (df['lat'] - seattle_lat)**2 + (df['long'] - seattle_long)**2
    )
    
    return df

# Apply feature engineering
print("Engineering features...")
train_engineered = engineer_features(train_df, is_train=True)
test_engineered = engineer_features(test_df, is_train=False)

print(f"\n✅ Original features: {train_df.shape[1]}")
print(f"✅ Engineered features: {train_engineered.shape[1]}")
print(f"✅ New features added: {train_engineered.shape[1] - train_df.shape[1]}")

# Show sample
print("\nSample of new features:")
new_cols = [col for col in train_engineered.columns if col not in train_df.columns]
print(train_engineered[new_cols].head())


Engineering features...

✅ Original features: 21
✅ Engineered features: 41
✅ New features added: 20

Sample of new features:
   price_per_sqft  price_per_lot  living_vs_neighbors  lot_vs_neighbors  \
0      148.421547      29.073918             1.089705          0.999892   
1      153.125000      87.876614             0.929692          0.773156   
2      116.279070      23.153508             0.919294          1.158530   
3      284.273387     499.998582             0.999194          0.938748   
4      181.250000      17.370470             0.804525          1.654609   

   has_basement  basement_ratio  quality_score  luxury_score  age  \
0             0        0.000000             21            10   54   
1             0        0.000000             28            11   23   
2             0        0.000000             24            11   21   
3             1        0.072581             21            10    6   
4             0        0.000000             21            10   21   

   years_

In [5]:
# Select features for modeling (exclude ID, date, target, and train-only features)
import os

train_only_features = ['price_per_sqft', 'price_per_lot']  # These only exist in train

feature_cols = [col for col in train_engineered.columns 
                if col not in ['id', 'date', 'price', 'zipcode'] + train_only_features]

X_train = train_engineered[feature_cols]
y_train = train_engineered['price']
X_test = test_engineered[feature_cols]

print(f"Features for modeling: {len(feature_cols)}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"\nFeature columns:\n{feature_cols}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrame
X_train_scaled = pd.DataFrame(X_train_scaled, columns=feature_cols, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=feature_cols, index=X_test.index)

# Save processed data
print("\n💾 Saving processed data...")
os.makedirs('../data/processed', exist_ok=True)

train_engineered.to_csv('../data/processed/train_engineered.csv', index=False)
test_engineered.to_csv('../data/processed/test_engineered.csv', index=False)

X_train_scaled.to_csv('../data/processed/X_train_scaled.csv', index=False)
X_test_scaled.to_csv('../data/processed/X_test_scaled.csv', index=False)
y_train.to_csv('../data/processed/y_train.csv', index=False, header=True)

# Save scaler
import pickle
with open('../data/processed/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

# Save feature names
with open('../data/processed/feature_names.txt', 'w') as f:
    f.write('\n'.join(feature_cols))

print("✅ All processed data saved to data/processed/")
print("\nSaved files:")
print("  - train_engineered.csv")
print("  - test_engineered.csv")
print("  - X_train_scaled.csv")
print("  - X_test_scaled.csv")
print("  - y_train.csv")
print("  - scaler.pkl")
print("  - feature_names.txt")


Features for modeling: 35
X_train shape: (16209, 35)
X_test shape: (5404, 35)

Feature columns:
['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'living_vs_neighbors', 'lot_vs_neighbors', 'has_basement', 'basement_ratio', 'quality_score', 'luxury_score', 'age', 'years_since_renovation', 'is_renovated', 'bedrooms_per_sqft', 'bathrooms_per_bedroom', 'above_ground_ratio', 'lot_coverage', 'has_view', 'high_grade', 'excellent_condition', 'location_cluster', 'dist_from_downtown']

💾 Saving processed data...
✅ All processed data saved to data/processed/

Saved files:
  - train_engineered.csv
  - test_engineered.csv
  - X_train_scaled.csv
  - X_test_scaled.csv
  - y_train.csv
  - scaler.pkl
  - feature_names.txt
